# Common Table Expressions (CTE)
- Simplify SQL-instructions
- Traverse recursively hierarchical and network structures


## 1. Give the PlayerNo and the amount of penalties **each** player has.

In [61]:
USE Tennis;

SELECT 
 Player.Name
,COUNT(Penalty.PlayerNo) AS [#Penalties]
FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
GROUP BY Player.Name

## Recap the `COUNT(...)` and `LEFT|RIGHT JOIN` statement

In [1]:
USE Tennis;

SELECT 
 Player.PlayerNo  AS 'Player - PlayerNo'
,Penalty.PlayerNo AS 'Penality - PlayerNo'
 FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo

In [97]:
USE Tennis;

SELECT 
 Player.PlayerNo
,COUNT(*)
FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
GROUP BY Player.PlayerNo

In [96]:
USE Tennis;

SELECT 
 Player.PlayerNo
,COUNT(Player.PlayerNo) -- -- Change is here.
FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
GROUP BY Player.PlayerNo

In [95]:
USE Tennis;

SELECT 
 Player.PlayerNo
,COUNT(Penalty.PlayerNo) -- Change is here.
FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
GROUP BY Player.PlayerNo

## 2. Give the total average of the amount of penalties each player has.
1. Using a `AVG` around a `COUNT(...)`
2. Using a subquery(Derived Table).
3. Using a view
4. Using a Common Table Expression (CTE)

In [70]:
-- 1. Using a AVG around a COUNT(*)
-- This won't work.
USE Tennis;

SELECT AVG(COUNT(Penalty.PlayerNo)) AS [#Penalties]
 FROM Players Player
    LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
GROUP BY Player.Name

In [98]:
-- 2. Using a subquery (Derived Table)
USE Tennis;

SELECT AVG(CAST(Fines.#Penalties AS DECIMAL)) AS 'Average Penalties Per Player' -- Without the cast it's a rounded INT.
FROM 
(
    SELECT COUNT(Penalty.PlayerNo) AS [#Penalties]
    FROM Players Player
        LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
    GROUP BY Player.PlayerNo -- What happens if... you GROUP BY the Name?
) AS Fines -- Do not forget an Alias for the subquery since it's used in the FROM clause of the outer query.

In [1]:
-- 3. Using a view.
USE Tennis;
GO
CREATE VIEW vw_amount_of_penalties_per_player AS
    SELECT COUNT(Penalty.PlayerNo) AS [#Penalties]
    FROM Players Player
        LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
    GROUP BY Player.PlayerNo
GO
SELECT AVG(CAST(#Penalties AS DECIMAL)) AS 'Average Penalties Per Player' -- Without the cast it's a rounded INT.
FROM vw_amount_of_penalties_per_player


In [2]:
-- 4. Using a Common Table Expression (CTE)

USE Tennis;

WITH Fines AS
(
    SELECT COUNT(Penalty.PlayerNo) AS [#Penalties]
    FROM Players Player
        LEFT JOIN Penalties Penalty ON Player.PlayerNo = Penalty.PlayerNo
    GROUP BY Player.PlayerNo
)
SELECT AVG(CAST(#Penalties AS DECIMAL))
FROM Fines;

## Views vs. CTE
- Similarities:
    - Both are virtual tables.
    - Data is not physically stored in a CTE nor VIEW.
    - Re-usability
- Differences:
    - CTE only exists during the SELECT statement
    - VIEW is stored as a Database Object

## Subquery vs. CTE
- Similarities:
    - Both are virtual tables.
    - Data is not physically stored in a CTE nor a subquery.
    - A subquery can easily be replaced by a CTE.
- Differences:
    - A CTE can be reused in the same query
    - Define:
        - Subquery is defined in the clause where it is used (SELECT/FROM/WHERE/…)
        - CTE is defined on top of the query since it's **WITH** the query.

## Keep it DRY (Don't Repeat Yourself)
Give the payment numbers and penalty amount that are not equal to the highest and lowest penalty ever paid by player 44. **Also show this highest and lowest amount in the result.**


In [13]:
USE tennis;

-- Subquery

-- SELECT 
--  Penalty.PaymentNo
-- ,Penalty.Amount
-- ,(SELECT MIN(amount) FROM penalties WHERE playerno = 44) AS 'Min'
-- ,(SELECT MAX(amount) FROM penalties WHERE playerno = 44) AS 'Max'
-- FROM penalties Penalty
-- WHERE Penalty.Amount <> (SELECT MIN(amount) FROM penalties WHERE playerno = 44)
--   AND Penalty.Amount <> (SELECT MAX(amount) FROM penalties WHERE playerno = 44);


-- Common Table Expression

WITH extrema AS 
(
    SELECT 
     MIN(Amount) AS 'Min'
    ,Max(Amount) AS 'Max'
    FROM Penalties  
    WHERE PlayerNo = 44
)
SELECT
 Penality.PaymentNo
,Penality.Amount
,extrema.[Min]
,extrema.[Max]
FROM Penalties Penality
    CROSS JOIN extrema
WHERE Penality.Amount <> extrema.[Min]
  AND Penality.Amount <> extrema.[Max]

## An xtreme example
Give per productclass the price of the cheapest product **and all products with that price**. 


In [0]:
USE xtreme;

WITH minimum_price_per_productclass AS
(
	SELECT 
	 Product.ProductClassID 
	,MIN(Product.Price) AS [Cheapest Price]
	FROM Product
	GROUP BY Product.ProductClassID
)
SELECT *
FROM Product
	JOIN minimum_price_per_productclass ON minimum_price_per_productclass.ProductClassID = Product.ProductClassID 
	                                   AND minimum_price_per_productclass.[Cheapest Price] = Product.Price 

## More than 1 WITH Component

Previous Example but having access to the maximum price

In [3]:
USE xtreme;

WITH 
minimum_price_per_productclass AS
(
	SELECT 
	 Product.ProductClassID 
	,MIN(Product.Price) AS [Cheapest Price]
	FROM Product
	GROUP BY Product.ProductClassID
),
maximum_price_per_productclass AS -- not used but possible!
(
	SELECT 
	 Product.ProductClassID 
	,MAX(Product.Price) AS [Cheapest Price]
	FROM Product
	GROUP BY Product.ProductClassID
)
SELECT *
FROM Product
	JOIN minimum_price_per_productclass ON minimum_price_per_productclass.ProductClassID = Product.ProductClassID 
	                                   AND minimum_price_per_productclass.[Cheapest Price] = Product.Price 

What is the total number of rows in both the penalties and the matches table?

In [14]:
USE tennis;

-- Without CTE
SELECT 
    (SELECT COUNT(*) FROM Penalties) + (SELECT COUNT(*) FROM Matches)

-- With CTE
WITH 
cte_pentalties AS 
(
	SELECT COUNT(*) AS 'Amount'
	FROM Penalties 
),
cte_matches AS 
(
	SELECT COUNT(*) AS 'Amount'
	FROM Matches 
)
SELECT (SELECT Amount from cte_pentalties) + (SELECT Amount from cte_matches)


# Recursion
Definition:
- We continue to execute a table expression until a condition is reached.

This allows you to solve problems like:
- Who are the friends of my friends etc. (in a social network)?
- What is the hierarchy of an organisation ? 
- Find the parts and subparts of a product (Bill of materials). 

Summary: 
- The 1st (non-recursive) expression is executed once and the 2nd expression is executed until it does not return any more results. 



In [0]:
-- The naive approach
SELECT 1 
UNION
SELECT 2 
UNION
SELECT 3 
UNION
SELECT 4 
UNION
SELECT 5 

In [20]:
-- CTE
WITH [Numbers] AS
(
    SELECT 1 AS Number
    UNION ALL
    SELECT Number + 1 
    FROM [Numbers]
    WHERE Number < 5
)
SELECT * FROM Numbers

In [23]:
-- CTE
WITH [Numbers] AS
(
    SELECT 1 AS Number
    UNION ALL
    SELECT Number + 1 
    FROM [Numbers]
    WHERE Number < 5
)
SELECT * FROM Numbers
OPTION(maxrecursion 2)

# Recursion - Tranversing a hierarchy
Give all employees who report directly or indirectly to Andrew Fuller (employeeid=2)


In [32]:
USE xtreme;
WITH Reports_to_Fuller AS 
(
    SELECT 
     Employee.EmployeeID
    ,Employee.FirstName
    ,Employee.LastName
    ,Employee.ReportsTo
    FROM Employee
    WHERE Employee.ReportsTo = 2

    UNION ALL

    SELECT 
     Employee.EmployeeID
    ,Employee.FirstName
    ,Employee.LastName
    ,Employee.SupervisorID
    FROM Employee
        JOIN Reports_to_Fuller ON Reports_to_Fuller.EmployeeID = Employee.ReportsTo
)
SELECT * FROM Reports_to_Fuller